# Import

In [1]:
import sys
# 캐시된 모듈 제거
for mod in list(sys.modules.keys()):
    if 'NPR' in mod or 'npr' in mod or 'networks' in mod:
        del sys.modules[mod]

# 이제 정상적으로 import
from model.NPR.npr_model import NPR

In [2]:
import sys
for mod in list(sys.modules.keys()):
    if any(x in mod for x in ['NPR', 'npr', 'networks', 'sgs', 'method']):
        del sys.modules[mod]

In [3]:
import os
from pathlib import Path 
from typing import Optional, Literal
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms

from utils.data.dataset import CorruptedDataset
from utils.visual.visualizer import DatasetVisualizer
from utils.eval.metrics import PredictionCollector, MetricsCalculator

# SGS method import
# from model.method.sgs import LGradSGS, SGSConfig
from model.LGrad.lgrad_model import LGrad

# GPU and Model select

In [4]:
!nvidia-smi

Mon Dec 29 15:06:25 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   37C    P0              33W / 250W |    260MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
DEVICE="cuda:1"
MODEL_LIST = ["lgrad", "npr"]
MODEL = MODEL_LIST[0]

# Dataloader

In [6]:
ROOT = "corrupted_dataset"
DATASETS = ["corrupted_test_data_biggan", "corrupted_test_data_crn", "corrupted_test_data_cyclegan", "corrupted_test_data_deepfake", "corrupted_test_data_gaugan", "corrupted_test_data_imle", "corrupted_test_data_progan", "corrupted_test_data_san", "corrupted_test_data_seeingdark", "corrupted_test_data_stargan", "corrupted_test_data_stylegan", "corrupted_test_data_stylegan2", "corrupted_test_data_whichfaceisreal"]
CORRUPTIONS = ["original", "gaussian_noise", "jpeg_compression"] # , "contrast", "fog", , "motion_blur", "pixelate"
if MODEL == "lgrad":
    transform=transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )
else:
    transform=transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )

In [7]:
dataset = CorruptedDataset(
    root= ROOT,
    datasets=DATASETS,
    corruptions=CORRUPTIONS,
    transform=transform
)

print(f"Total samples: {len(dataset)}")

Total samples: 270987


# Model load

In [8]:
from model.LGrad.lgrad_model import LGrad
from model.NPR.npr_model import NPR

#LGrad
STYLEGAN_WEIGHTS_ROOT="model/LGrad/weights/karras2019stylegan-bedrooms-256x256_discriminator.pth"
CLASSIFIER_WEIGHTS_ROOT="model/LGrad/weights/LGrad-Pretrained-Model/LGrad-4class-Trainon-Progan_car_cat_chair_horse.pth"

#NPR
NPR_WEIGHTS_ROOT="model/NPR/weights/NPR.pth"

if MODEL == "lgrad":
    model = LGrad(
        stylegan_weights=STYLEGAN_WEIGHTS_ROOT,
        classifier_weights=CLASSIFIER_WEIGHTS_ROOT,
        device=DEVICE
    )
    model_name="LGrad"
    LAMBDA=0.03
    DELTA=0.01
    ITERATIONS=5
    STEP_SIZE=0.1
elif MODEL == "npr":
    model = NPR(
        weights=NPR_WEIGHTS_ROOT,
        device=DEVICE
    )
    model_name="NPR"
    LAMBDA=0.005
    DELTA=0.01
    ITERATIONS=3
    STEP_SIZE=0.1


/workspace/robust_deepfake_ai/model/LGrad/lgrad_model.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(stylegan_weights, map_location="cpu"),
/workspace/robus

In [9]:
from model.method.sgs import SGSConfig, UnifiedSGS
from model.method.sas import UnifiedSAS, SASConfig
from model.method.sgsv2 import UnifiedSGSv2, SGSv2Config
# sgs_config = SGSConfig(
#     K=4,
#     model=model_name,
#     denoise_target="artifact",
#     huber_tv_lambda=LAMBDA,
#     huber_tv_delta=DELTA,
#     huber_tv_iterations=ITERATIONS,
#     huber_tv_step_size=STEP_SIZE,

#     device=DEVICE,
# )

# sas_config = SASConfig(
#     K=5,
#     model=model_name,
#     denoise_target="artifact",
#     lambda_range=(0.3, 2.0),
#     jitter_strength=0.2,

#     # Huber-TV
#     huber_tv_lambda=0.05,
#     huber_tv_delta=0.01,

    
#     fusion_scale=2.0,
#     device=DEVICE,
# )

# sasv2_config = SGSv2Config(
#     K=5,
#     model=model_name,
#     denoise_target="artifact",

#     fusion_alpha=0.3,
#     beta_threshold=0.02,
#     beta_softness=0.01,

#     device=DEVICE
# )

# model = UnifiedSGSv2(model, sasv2_config) # 실제로는 SASv2이지만 이름 잘못 설정함

# model.model.eval()

In [10]:
# from model.method.sasv3 import UnifiedSASv3, SASv3Config

# config = SASv3Config(
#     K=5,
#     model="LGrad",
#     denoise_target="artifact",
#     fusion_method="robust",
#     fusion_scale=2.0,
#     adaptive_beta=True,
#     beta_method="auto",  # Calibration으로 자동
#     device=DEVICE
# )

# model = UnifiedSASv3(model, config)
# model.model.eval()

In [11]:
# from model.method import UnifiedSASv4, SASv4Config

# config = SASv4Config(
#     K=5,
#     model="LGrad",
#     denoise_target="artifact",
#     residual_beta=1.0,
#     dmap_percentile=95.0,
#     device=DEVICE,
# )

# model = UnifiedSASv4(model, config)

In [12]:
# from model.method import UnifiedSASv5, SASv5Config, create_lgrad_sasv5

# config = SASv5Config(
#     K=3,
#     model="LGrad",
#     sigmas=[0.0, 0.8, 1.6],  # Gaussian blur σ
#     denoise_target="artifact",  # GoG!
#     residual_beta=1.0,
#     device=DEVICE,
# )

# model = UnifiedSASv5(model, config)

In [13]:
# v5, v4를 위함
# from torch.utils.data import DataLoader, Subset
# progan_clean_indices = [
#     i for i, s in enumerate(dataset.samples)
#     if s['dataset'] == "corrupted_test_data_progan" and s['corruption'] == "original"
# ]

# print(f"ProGAN clean samples: {len(progan_clean_indices)}")

# # ===== 5. Subset & DataLoader 생성 =====
# clean_subset = Subset(dataset, progan_clean_indices)
# clean_loader = DataLoader(
#     clean_subset,
#     batch_size=32,
#     shuffle=False,
#     num_workers=4,
#     drop_last=False  # Calibration은 drop_last=False
# )

# model.calibrate_disagreement_threshold(clean_loader, percentile=95.0)

In [14]:
from model.method import UnifiedSASv6, SASv6Config

config = SASv6Config(
    model="LGrad",
    gaussian_sigma=0.8,
    dct_threshold=2.5,
    dwt_threshold=0.05,
    # ===== DWT 비활성화 =====
    apply_dwt_to_input=False,     # I_d에 DWT 적용 안함
    apply_dwt_to_residual=False,  # R_d에 DWT 적용 안함
    input_mode="I_d_wave",
    # residual_type="R_d",
    device=DEVICE
)
model = UnifiedSASv6(model, config)

[SASv6] Initialized for LGrad
[SASv6] Gaussian: kernel=3, σ=0.8
[SASv6] Block DCT: thresh=2.5, clip=10.0
[SASv6] DWT: type=haar, λ=0.05
[SASv6] Input mode: I_d_wave
[SASv6] Residual type: R_d


In [15]:
from torchsummary import summary
summary(model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
          Identity-1          [-1, 3, 256, 256]               0
          Identity-2          [-1, 3, 256, 256]               0
          Identity-3         [-1, 64, 256, 256]               0
         LeakyReLU-4         [-1, 64, 256, 256]               0
         ConvBlock-5         [-1, 64, 256, 256]             256
          Identity-6         [-1, 64, 256, 256]               0
          Identity-7         [-1, 64, 256, 256]               0
          Identity-8         [-1, 64, 256, 256]               0
         LeakyReLU-9         [-1, 64, 256, 256]               0
        ConvBlock-10         [-1, 64, 256, 256]          36,928
         Identity-11         [-1, 64, 256, 256]               0
        BlurLayer-12         [-1, 64, 256, 256]               0
         Identity-13        [-1, 128, 128, 128]               0
        LeakyReLU-14        [-1, 128, 1

In [16]:
from torch.utils.data import Subset, DataLoader
import random

# Evaluation
calc = MetricsCalculator()
all_results = {}

for dataset_name in DATASETS:
    for corruption in CORRUPTIONS:
        combination_indices = [
            i for i, s in enumerate(dataset.samples)
            if s['dataset'] == dataset_name and s['corruption'] ==corruption
        ]

        if len(combination_indices) == 0:
            print(f"{dataset_name}-{corruption}: 샘플 없음, 스킵")
            continue

        print(f"\n{'='*60}")
        print(f"평가 중: {dataset_name}-{corruption}")
        print(f"샘플 수: {len(combination_indices)}")
        print(f"{'='*60}")

        # Subset과 DataLoader 생성
        subset = Subset(dataset, combination_indices)
        dataloader = DataLoader(
            subset,
            batch_size=16,
            shuffle=False,
            num_workers=4,
            # collate_fn=collate_fn,
            drop_last=True
        )

        # 평가
        metrics = calc.evaluate(
            model=model,
            dataloader=dataloader,
            device=DEVICE,
            name=f"{dataset_name}-{corruption}"
        )

        # 즉시 결과 출력
        print(f"\n결과:")
        print(f"  Accuracy: {metrics['accuracy']*100:.2f}%")
        print(f"  AUC:      {metrics['auc']*100:.2f}%")
        print(f"  AP:       {metrics['ap']*100:.2f}%")
        print(f"  F1:       {metrics['f1']*100:.2f}%")

        # 결과 저장
        all_results[(dataset_name, corruption)] = metrics

# 전체 결과 테이블 출력
print(f"\n\n{'='*60}")
print("전체 결과 요약")
print(f"{'='*60}\n")
calc.print_results_table()
calc.summarize_by_corruption(all_results)
calc.summarize_by_dataset(all_results)


평가 중: corrupted_test_data_biggan-original
샘플 수: 4000


corrupted_test_data_biggan-original: 100%|██████████| 250/250 [00:47<00:00,  5.23it/s]



결과:
  Accuracy: 49.95%
  AUC:      46.56%
  AP:       47.39%
  F1:       1.86%

평가 중: corrupted_test_data_biggan-gaussian_noise
샘플 수: 4000


corrupted_test_data_biggan-gaussian_noise: 100%|██████████| 250/250 [00:47<00:00,  5.25it/s]



결과:
  Accuracy: 50.05%
  AUC:      50.55%
  AP:       51.59%
  F1:       0.20%

평가 중: corrupted_test_data_biggan-jpeg_compression
샘플 수: 4000


...


결과:
  Accuracy: 50.20%
  AUC:      49.31%
  AP:       49.43%
  F1:       0.10%

평가 중: corrupted_test_data_stargan-jpeg_compression
샘플 수: 3998


corrupted_test_data_stargan-jpeg_compression: 100%|██████████| 249/249 [00:47<00:00,  5.23it/s]



결과:
  Accuracy: 50.15%
  AUC:      51.08%
  AP:       51.20%
  F1:       0.30%

평가 중: corrupted_test_data_stylegan-original
샘플 수: 11982


corrupted_test_data_stylegan-original:  27%|██▋       | 200/748 [00:38<01:45,  5.19it/s]


KeyboardInterrupt: 